Análisis de sentimiento. Implementa un módulo de detección de sentimiento de los posts. 

Para entrenar y evaluar diferentes métodos, dispones de la columna “sentiment” en el dataset.  
Al  igual  que  en  el  módulo  de  clasificación  de  subreddits,  Para  evaluar  cada método, se utilizará la métrica f1 score, y se utilizará un 70% de los datos del dataset para entrenamiento y un 30% para test (realizando un sampling aleatorio previo). Deberás probar 
y evaluar los siguientes métodos: 

- Un método basado en lexicons 
- Un método basado en palabras únicas en textos de cada tipo de sentimiento 
- Un método basado en Word embeddings + algoritmo de machine learning de clasificación.

La función de este módulo tendrá como nombre sentiment_analysis(text: str)

### Método basado en lexicons

In [ ]:
def sentiment_analysis_lexicons(text: str):
    

In [ ]:
def sentiment_analysis(text:str):
    pass